In [13]:
from typing_extensions import List, TypedDict, Literal
from langchain_core.documents import Document
from langgraph.graph import StateGraph, START, END
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 1. 상태(State) 정의
class AgentState(TypedDict):
    query: str
    context: List[Document]
    answer: str
    tax_base_equation: str
    tax_deduction: str
    market_ratio: str
    tax_base: str

# 2. 필요한 함수/노드 정의
llm = ChatOpenAI(model='gpt-4o', temperature=0)

router_prompt = ChatPromptTemplate.from_messages([
    ('system', 
        """
        사용자의 질문이 '세금'에 대한 것인지, '부동산'에 대한 것인지 분류해 주세요. 
        만약 둘 다 아니라면 'irrelevant'라고 응답하세요. 
        '세금', '부동산', 'irrelevant' 중 하나를 선택하세요.
        """
    ),
    ('human', '{query}')
])

def router(state: AgentState) -> Literal['tax', 'real_estate', 'irrelevant']:
    query = state['query']
    router_chain = router_prompt | llm | StrOutputParser()
    response = router_chain.invoke({'query': query})
    if '세금' in response:
        return 'tax'
    elif '부동산' in response:
        return 'real_estate'
    else:
        return 'irrelevant'

# 부동산 관련 함수
def get_tax_base_equation(state: AgentState) -> AgentState:
    tax_base_equation = "과세표준 = (공시가격 합산 - 공제액) × 공정시장가액비율"
    return {"tax_base_equation": tax_base_equation}

def get_tax_deduction(state: AgentState) -> AgentState:
    tax_deduction = "주택에 대한 종합부동산세 계산 시, 1세대 1주택자의 경우 공제금액은 12억 원입니다. 법인 또는 법인으로 보는 단체는 6억 원, 그 외의 경우는 9억 원이 공제됩니다."
    return {"tax_deduction": tax_deduction}

def get_market_ratio(state: AgentState) -> AgentState:
    market_ratio = "2025년 주택 공시가격에 대한 공정시장가액비율은 100%입니다."
    return {"market_ratio": market_ratio}

def calculate_tax_base(state: AgentState) -> AgentState:
    tax_base = "26억 원"
    return {"tax_base": tax_base}

def calculate_tax_rate(state: AgentState) -> AgentState:
    tax_rate = f"과세표준 {state['tax_base']}에 대한 세율을 계산하여 answer를 반환합니다."
    return {"answer": tax_rate}

# 세금 관련 함수
def retrieve(state: AgentState) -> AgentState:
    print("세금 정보를 검색합니다.")
    return {"context": [Document(page_content="연봉 5천만원에 대한 세금은...")]}

def generate(state: AgentState) -> AgentState:
    print("답변을 생성합니다.")
    return {"answer": f"답변 생성: {state['context'][0].page_content}"}

def rewrite(state: AgentState) -> AgentState:
    print("답변을 재작성합니다.")
    return {"answer": "재작성된 답변입니다."}

def check_doc_relevence(state: AgentState) -> Literal['relevant', 'irrelevant']:
    print("문서 관련성을 확인합니다.")
    return 'relevant'

def check_hallucination(state: AgentState) -> Literal['not hallucinated', 'hallucinated']:
    print("환각을 확인합니다.")
    return 'not hallucinated'

def check_helpfulness_grader(state: AgentState) -> Literal['helpful', 'unhelpful']:
    print("도움이 되는지 평가합니다.")
    return 'helpful'

def check_helpfulness(state: AgentState) -> AgentState:
    return {"answer": "최종 답변: 안녕하세요!"}

# 3. 그래프 구축
builder = StateGraph(AgentState)

builder.add_node('router', router)
builder.add_node('get_tax_base_equation', get_tax_base_equation)
builder.add_node('get_tax_deduction', get_tax_deduction)
builder.add_node('get_market_ratio', get_market_ratio)
builder.add_node('calculate_tax_base', calculate_tax_base)
builder.add_node('calculate_tax_rate', calculate_tax_rate)
builder.add_node('retrieve', retrieve)
builder.add_node('generate', generate)
builder.add_node('rewrite', rewrite)

builder.add_edge(START, 'router')

# 수정된 부분: add_conditional_edges의 두 번째 인자를 노드 함수 이름으로 변경
builder.add_conditional_edges(
    'router',
    router,
    {
        'real_estate': 'get_tax_base_equation',
        'tax': 'retrieve',
        'irrelevant': END
    }
)

builder.add_edge('get_tax_base_equation', 'get_tax_deduction')
builder.add_edge('get_tax_deduction', 'get_market_ratio')
builder.add_edge('get_market_ratio', 'calculate_tax_base')
builder.add_edge('calculate_tax_base', 'calculate_tax_rate')
builder.add_edge('calculate_tax_rate', END)

# 수정된 부분: check_doc_relevence를 노드로 추가 후, retrieve 노드 다음에 연결
builder.add_node('check_doc_relevence', check_doc_relevence)
builder.add_conditional_edges(
    'retrieve',
    check_doc_relevence,
    {
        'relevant': 'generate',
        'irrelevant': END
    }
)

# 수정된 부분: check_hallucination을 노드로 추가 후, generate 노드 다음에 연결
builder.add_node('check_hallucination', check_hallucination)
builder.add_conditional_edges(
    'generate',
    check_hallucination,
    {
        'not hallucinated': END,  # 도움이 된다고 가정하고 바로 종료
        'hallucinated': 'generate'
    }
)

# rewrite 노드에서 retrieve로 돌아오는 엣지 추가
builder.add_edge('rewrite', 'retrieve')

# 4. 그래프 컴파일
graph = builder.compile()

# 5. 테스트 코드
print("--- 세금 관련 질문 테스트 ---")
inputs_tax = {"query": "연봉 5천만원 세금은 얼마인가요?"}
for s in graph.stream(inputs_tax):
    print(s)
    print("---")

print("\n--- 부동산 관련 질문 테스트 ---")
inputs_real_estate = {"query": "주택 종부세 계산 방법을 알려주세요."}
for s in graph.stream(inputs_real_estate):
    print(s)
    print("---")

--- 세금 관련 질문 테스트 ---


InvalidUpdateError: Expected dict, got tax
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE